In [0]:
stg_posts_df = spark.table("silver.stg_posts")
raw_users_df = spark.table("bronze.raw_users")

In [0]:
from pyspark.sql import DataFrame
import pyspark.sql.functions as F

def posts_users_OBT(stg_posts_df: DataFrame, raw_users_df: DataFrame) -> DataFrame:
    return (
        stg_posts_df
        .alias("posts")
        .withColumnRenamed("CreationDate", "PostCreationDate")
        .join(
            other=raw_users_df
                .withColumnRenamed("CreationDate", "UserCreationDate")
                .alias("users"),
            on=F.col("posts.OwnerUserId") == F.col("users.Id"),
            how="left"
        )
    )

marts_posts_user_df = posts_users_OBT(stg_posts_df, raw_users_df)

In [0]:
marts_posts_user_df.write.mode("overwrite").saveAsTable("gold.marts_posts_users")

In [0]:
display(spark.table("gold.marts_posts_users").limit(3))